In [1]:
import pylab
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table

/Users/pdh21/anaconda3/envs/xidplus/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
import seaborn as sns

This notebook uses all the raw data from the XID+MIPS catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in XID+MIPS catalogue

In [3]:
XID_MIPS=Table.read('../../../data/Scudderetal2016/all_sources.fits')

In [4]:
XID_MIPS[0:10]

Fields,ra,dec
bytes5,float64,float64
10932,150.624988,2.278824
10932,150.626222,2.279251
10932,150.623847,2.278892
10932,150.626046,2.280238
10932,150.623673,2.280668
10932,150.626977,2.282143
10932,150.626327,2.282569
10932,150.622767,2.283329
10932,150.620387,2.283089


## Read in Maps

In [5]:

pswfits='/Volumes/pdh_storage/dmu_products/dmu19/dmu19_HELP-SPIRE-maps/data/COSMOS_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits='/Volumes/pdh_storage/dmu_products/dmu19/dmu19_HELP-SPIRE-maps/data/COSMOS_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits='/Volumes/pdh_storage/dmu_products/dmu19/dmu19_HELP-SPIRE-maps/data/COSMOS_SPIRE500_v1.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [6]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

In [11]:
## Set XID+ prior class

In [7]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(XID_MIPS['ra'],XID_MIPS['dec'],'Scudder2016.fits')#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu)
prior350.prior_cat(XID_MIPS['ra'],XID_MIPS['dec'],'Scudder2016.fits')
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu)
prior500.prior_cat(XID_MIPS['ra'],XID_MIPS['dec'],'Scudder2016.fits')
prior500.prior_bkg(-5.0,5)

In [8]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [10]:
import pickle
#from moc, get healpix pixels at a given order

output_folder='.../../../data/Scudderetal2016/'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'version':xidplus.io.git_version()},outfile)


writing total_bytes=45186307...
writing bytes [0, 45186307)... done.


Create MOCs based on all sources

In [25]:
from astropy.coordinates import SkyCoord
from astropy import units as u
for i in np.unique(XID_MIPS['Fields']):
    c = SkyCoord(ra=XID_MIPS[XID_MIPS['Fields']==i]['ra']*u.degree,dec=XID_MIPS[XID_MIPS['Fields']==i]['dec']*u.degree)
    moc=pymoc.util.catalog.catalog_to_moc(c,18,14)
    moc.write('.../../../data/Scudderetal2016/MOCS/MOC_{}.fits'.format(i),overwrite=True)